In [13]:
import sys
sys.path.append('../')
import torch
from tools.utils import manual_seed
import PIL
import PIL.Image

import numpy as np
import matplotlib.pyplot as plt
from tools.img import rgb2yuv, yuv2rgb, rgb_to_yuv, yuv_to_rgb
from torchvision.transforms.transforms import ToTensor
from tools.dataset import get_dataloader, get_dataset_class_and_scale, get_dataset_normalization, get_de_normalization
from omegaconf import OmegaConf
from tools.img import tensor2ndarray, ndarray2tensor
from torchmetrics import StructuralSimilarityIndexMeasure
from tools.dataset import get_dataloader
# from tools.inject_backdoor import patch_trigger

manual_seed(42)
loss = StructuralSimilarityIndexMeasure

In [ ]:
target_folder = '../' + 'results/imagenette/inba/20241018041919_ab4_best'
path = f'{target_folder}/config.yaml'
config = OmegaConf.load(path)
manual_seed(config.seed)

device = f'cuda:{config.device}'
num_class, scale = get_dataset_class_and_scale(config.dataset_name)
if config.model == "resnet18":
    from models.preact_resnet import PreActResNet18
    net = PreActResNet18(num_classes=num_class).to(f'cuda:{config.device}')
elif config.model == "rnp":
    from models.resnet_cifar import resnet18
    net = resnet18(num_classes=num_class).to(f'cuda:{config.device}')
elif config.model == "repvgg":
    from repvgg_pytorch.repvgg import RepVGG
    net = RepVGG(num_blocks=[2, 4, 14, 1], num_classes=num_class, width_multiplier=[1.5, 1.5, 1.5, 2.75]).to(device=f'cuda:{config.device}')
else:
    raise NotImplementedError(config.model)
ld = torch.load(f'{target_folder}/results.pth', map_location=device)
net.load_state_dict(ld['model'])
net.to(device)

Load a example image and plot RGB channels.

Convert the RGB image to YUV channels, and plot them.

Use INBA to add pertubation to the imaginary part of U channel after FFT.

In [ ]:
_, dl = get_dataloader(config.dataset_name, config.batch, config.pin_memory, config.num_workers)
x_c = None
for batch, label in dl:
    for i in range(batch.shape[0]):
        if label[i] == 1:
            x_c = batch[i]
            break
    if x_c != None:
        break

sys.path.append('./run')
sys.path.append(target_folder)
from inject_backdoor import patch_trigger
x_p = patch_trigger(x_c, config)

x_c = x_c.to(device)
x_p = x_p.to(device)
x_c = get_de_normalization(config.dataset_name)(x_c).squeeze()
x_p = get_de_normalization(config.dataset_name)(x_p).squeeze()
x_p = torch.clip(x_p, 0, 1)

x_c = get_dataset_normalization(config.dataset_name)(x_c)
x_p = get_dataset_normalization(config.dataset_name)(x_p)

if config.model == 'repvgg':
    net.deploy = True
net.eval()
with torch.no_grad():
    out_c = net(x_c.unsqueeze(0))
    out_p = net(x_p.unsqueeze(0))
    _, y_c = torch.max(out_c, 1)
    _, y_p = torch.max(out_p, 1)

print(y_c.item())
print(y_p.item())

print(x_c[0, 0:3, 0:3])
print(x_p[0, 0:3, 0:3])

x_c = get_de_normalization(config.dataset_name)(x_c).squeeze()
x_p = get_de_normalization(config.dataset_name)(x_p).squeeze()

_, ax = plt.subplots(1, 2, figsize=(15, 10))
ax[0].imshow(tensor2ndarray(x_c))
ax[0].axis('off')
ax[1].imshow(tensor2ndarray(x_p))
ax[1].axis('off')
plt.show()
ssim_metric = loss(data_range=1.0).to(device)(x_c.unsqueeze(0), x_p.unsqueeze(0))
print(f'ssim: {ssim_metric}')